### Análisis exploratorio - EDA

#### Importación de librerías

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#### Importación de datos

In [12]:
df = pd.read_csv("https://raw.githubusercontent.com/jaimesz11/Ataques_al_corazon-Predicciones_y_analisis/main/Datos/heart.csv?token=ARVDOLPNHQVTMOGJMTYZXYLAVVKBO")
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
